> __В программировании по этикету необходимо сбрасывать ядро со всеми выводами при передаче своего notebook.__
> __В моем коде присутствуют стандартные библиотеки, поэтому все они легко импортируются и запускаются. Если же возникли проблемы при запуске импортирования, то можно раскоментить блок ниже и скачать все необходимые библиотеки :)__

In [ ]:
# !pip install catboost
# !pip install sklearn
# !pip install string
# !pip install seaborn
# !pip install matplotlib
# !pip install numpy
# !pip install pandas

## Импортирование необходимых для работы библиотек

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import string
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report,roc_auc_score

## Работа с датасетом

In [ ]:
df = pd.read_csv('passwords.csv')

In [ ]:
# Выведем первые 5 строк сета, чтобы всегда можно было посмотреть на датасет
df.head()

In [ ]:
# Проверяем есть ли пропущнные значения в паролях
print(f"Наличие пропущенных данных в паролях: {len(df[df.password == ''])}")

---
1) ***Мы увидели, что в паролях нет пропущенных значений, поэтому нам не нужно никак заполнять пропуски или исключать эти строки из выборки***
2) ***Далее необходимо создать дополнительные признаки из паролей, чтобы модель, смогла обучиться на закономерностях этих особенностей:***
*  **Создаем функцию, которая делит наш пароль на признаки для обучения модели:**
    1. ***Length (Длина):*** Этот признак представляет собой длину пароля, выраженную в количестве символов. Он указывает на общее количество символов в пароле.
    2. ***Uppercase (Заглавные буквы):*** Этот признак представляет собой количество букв верхнего регистра (заглавных букв) в пароле. Он показывает, сколько букв верхнего регистра содержится в пароле.
    3. ***Lowercase (Строчные буквы):*** Этот признак представляет собой количество букв нижнего регистра (строчных букв) в пароле. Он показывает, сколько букв нижнего регистра содержится в пароле.
    4. ***Digit (Цифры):*** Этот признак представляет собой количество цифр в пароле. Он указывает, сколько цифр содержится в пароле.
    5. ***Special Character (Специальные символы):*** Этот признак представляет собой количество специальных символов (знаки пунктуации и другие символы) в пароле. Он показывает, сколько таких символов присутствует в пароле.
    6. ***Complexity (Сложность):*** Этот бинарный признак указывает на сложность пароля. Он равен 1, если пароль содержит как минимум 2 заглавные буквы, 3 строчные буквы, 2 цифры и 1 специальный символ; в противном случае, он равен 0.
    7. ***Unique Characters (Уникальные символы):*** Этот признак представляет собой количество уникальных символов в пароле. Он указывает, сколько различных символов используется в пароле.
---

In [ ]:
def generate_password_features(passwords_df):
    
    features = []
    
    for password, password_class in zip(passwords_df["password"], passwords_df["strength"]):
        length_feature = len(password)
        uppercase_feature = sum(1 for c in password if c.isupper())
        lowercase_feature = sum(1 for c in password if c.islower())
        digit_feature = sum(1 for c in password if c.isdigit())
        special_character_feature = sum(1 for c in password if c in string.punctuation)
        
        complexity_feature = int(uppercase_feature >= 2 and lowercase_feature >= 3 and digit_feature >= 2 and special_character_feature >= 1)
        
        unique_characters_feature = len(set(password))
        
        features.append([password,length_feature, uppercase_feature, lowercase_feature, digit_feature, special_character_feature, complexity_feature, unique_characters_feature, password_class])
    
    feature_names = ["password","Length", "Uppercase", "Lowercase", "Digit", "Special Character", "Complexity", "Unique Characters", "strength"]
    
    df = pd.DataFrame(features, columns=feature_names)
    
    return df

In [ ]:
df_features = generate_password_features(df)

In [ ]:
df_features.head()

In [ ]:
# Отсортируем датафрейм по увелечению длины
df_features.sort_values(by = 'Length').head()

---
- ***Рассмотрим случаи, когда длина пароля состоит из одной буквы/цифры или пары символов.Как минимум это странно и врядли это можно назвать паролем,поэтому такие случаи следовало бы убирать. <font color='green'>Каким образом?</font>. На большинстве сайтов требуемая длинна пароля должна варироваться от 6 до 12 символов, поэтому следует ограничить длину пароля нижней границей -  <font color='red'>6 символами</font>***
- ***Также хочется заметить,что в паролях содержатся недопустимые символы, однако в зависимости от политики различных сайтов ограничения на специальные символы в паролях могут отличаться. Можно зайти [на сайт Сбербанка](https://www.sberbank.ru/ru/person/kibrary/vocabulary/parol?TSPD_101_R0=08fbdc5594ab20003bbbc7801b516e88fe62422f984e380f12ef85221e4a714846f3b77dc814ba760811aa68ce14300013c8fa5830c4b64fbd137d1937ecd0dd4d558099986b01d9b56229b920c1c676ec8bad3b210db649d745dca43049b365) и посмотреть какие там требования к паролю.***

> <font color='red'>***Это были бы весьма логичные замечания после проделанного анализа. Однако на следующем графике можно будет увидеть, что вариация классов сильно зависит от длины пароля. Корректировка этого признака будет ухудшать обучение модели и нарушать зависимость с регрессантом, поэтому этот признак оставляем без изменений,поскольку он будет вносить большой импакт в обучение модели.***</font>

- ***На графике можно четко увидеть как происходит деление паролей на классы в зависимости от длины:***
    1. ***0 класс - длина пароля состовляет от 1 до 7 символов***
    2. ***1 класс - длина пароля состовляет от 8 до 13 символов***
    3. ***2 класс - длина пароля начинатется от 14 символов***
    
<font color='red'>***Показатели класса пароля от длины представлены с учетом выбросов на графике****</font>

--- 

In [ ]:
# Создание boxplot
plt.figure(figsize=(10, 10))

# Создание клетчатого фона
ax = plt.gca()
ax.set_facecolor('white')
plt.grid(color='black', linestyle='--', linewidth=0.5)

# Сортировка классов и цвета
class_order = sorted(df_features['strength'].unique())
class_colors = plt.cm.viridis(np.linspace(0, 1, len(class_order)))

boxplot_data = [df_features[df_features['strength'] == class_val]['Length'] for class_val in class_order]
box = plt.boxplot(boxplot_data, labels=class_order, patch_artist=True)

# Установка цветов для boxplot
for patch, color in zip(box['boxes'], class_colors):
    patch.set_facecolor(color)

# Добавление названия осей и самого графика
plt.yticks(np.arange(0, max(df_features['Length']) + 1, 1))
plt.xlabel('Классы паролей')
plt.ylabel('Длина пароля')
plt.title('Boxplot длин паролей по классам')
plt.show()

## Создание экземпляра и обучение модели на предобработанных данных

## Основной способ

> ***Как было сказана выше, что вся закономерность классов паролей зависит,в большинстве своем, от длины пароля, поэтому для нашей задачи можно не перебирать модели для обучения и взять модель классификации из личных предпочтнений. На этом датасете хорошо отработают модели,основой которых является стратегия бустинга, а также алгоритмы основанные на решающих деревьях, поэтому нет особой строгости при выборе модели для этой задачи.***

- **Буду использовать CatBoostClassifier в этой задаче классификации за счет его преимуществ:**
- [ ] CatBoost может обучаться с высокой скоростью и требует меньше предварительной настройки параметров.
- [ ] CatBoost имеет встроенные механизмы для борьбы с переобучением и автоматической остановки обучения, когда происходит переобучение.
- [ ] CatBoost может быть эффективным на больших и сложных наборах данных.

In [ ]:
# Посмотрим на баланс наших классов в датасете
df_features.strength.value_counts()

- - -

> - **По статистике видно, что имеется дисбаланс классов в наших данных,поэтому следует применять <font color='red'>***Recall,Precision и F1***</font>.**
> - **<font color='red'>***Accuracy***</font> простая и интуитивная метрика, которая измеряет общую точность классификации, но она подходит для сбалансированных классов.**

- __Точность (Precision)__- фокусируется на точности предсказаний положительного класса
- __Полнота (Recall)__ - фокусируется на способности модели обнаруживать положительные примеры
- __F1-мера (F1-Score)__ - это гармоническое среднее между точностью и полнотой, что делает ее хорошим компромиссом при дисбалансе классов.
- - -

In [ ]:
#Используем default настройки модели CatBoostClassifier и не делаем тюнинг гиперпараметров, тк у нас не большой объем данных 
#и существует явная зависимость между регессоров(длина) и регрессантом
cat = CatBoostClassifier(loss_function='MultiClass',verbose=False)

In [ ]:
X = df_features[['Length', 'Uppercase', 'Lowercase', 'Digit', 'Special Character','Complexity', 'Unique Characters']]
y = df_features.strength
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25)

In [ ]:
cat.fit(X_train,y_train,verbose = False)

In [ ]:
y_pred = cat.predict(X_test)

In [ ]:
#Здесь average='weighted' вычислит взвешенную precision, что учитывает разные размеры классов в мультиклассовой задаче. 
precision = precision_score(y_test, y_pred,average='weighted')
recall = recall_score(y_test, y_pred,average='weighted')
f1 = f1_score(y_test, y_pred,average='weighted')
# Можно сделать классификационну матрицу, которая также покажет нам показатели наших метрик
report = classification_report(y_test, y_pred)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(report)

## Альтернативный способ решения

> __Как отмечалось выше, существует явная зависимость длины пароля на качество пароля,поэтому альтернативным решением этого задания будет функция,которая также будет соотвествовать требованиями задания, поэтому можно не создавать полноценный pipeline ML задачи. Условия по ограничению длины пароля я беру с Boxplot. Пароль с 0 кол-вом символом существовать не может, поэтому строго ограничиваю нижнюю границу для "0" класса пароля.__


In [ ]:
def predict_without_ml_model(passwords):
    predictions = []
    
    if isinstance(passwords, str):
        # Если передан один пароль,то преобразуем его в список, чтобы функция сделала необходимые преобразования
        passwords = [passwords]
        
    for password in passwords:
        if len(password) > 0 and len(password) < 8  :
            predictions.append(0)
            
        elif len(password) >= 8 and len(password) < 14:
            predictions.append(1)
            
        elif len(password) >= 14:
            predictions.append(2)
            
        elif len(password) == 0:
            return('Password with these conditions is not available')
            
    return predictions

In [ ]:
X = df.password
y = df.strength

In [ ]:
pred = predict_without_ml_model(X)

In [ ]:
#Смотрим на показатели метрики и делаем отчет нашей классификации,чтобы смотреть все измерения метрик в одном блоке
precision = precision_score(y, pred, average='weighted')
recall = recall_score(y, pred, average='weighted')
f1 = f1_score(y, pred, average='weighted')
report = classification_report(y, pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print('')
print(report)

## Блок для обучения pipeline и предсказания класса новых паролей

--- 
* ***В этом блоке записываем нашу функцию из блоков выше и контейнер, в котором будут трансформер для наших данных и модель для обучения***
* ***Используем Pipeline, так как  это позволяет нам сразу предобрабатывать входные данные и пускать их в input модели***

In [ ]:
def extract_numeric_password_features(passwords):
    
    features = []
    
    if isinstance(passwords, str):
        # Если передан один пароль,то преобразуем его в список, чтобы функция сделала необходимые преобразования
        passwords = [passwords]
        
    for password in passwords:
            
        length_feature = len(password)
        uppercase_feature = sum(1 for c in password if c.isupper())
        lowercase_feature = sum(1 for c in password if c.islower())
        digit_feature = sum(1 for c in password if c.isdigit())
        special_character_feature = sum(1 for c in password if c in string.punctuation)

        complexity_feature = int(uppercase_feature >= 2 and lowercase_feature >= 3 and digit_feature >= 2 and special_character_feature >= 1)

        unique_characters_feature = len(set(password))

        features.append([length_feature, uppercase_feature, lowercase_feature, digit_feature, special_character_feature, complexity_feature, unique_characters_feature])

    feature_names = ["Length", "Uppercase", "Lowercase", "Digit", "Special Character", "Complexity", "Unique Characters"]
    
    df = pd.DataFrame(features, columns=feature_names)
    
    return df

# Создание трансформера для признаков
password_feature_extractor = FunctionTransformer(func=extract_numeric_password_features, validate=False)

# Создание пайплайна
#В блоке обучения была описана причина выбора default параметров модели
pipeline = Pipeline([
    ('feature_extraction', FunctionTransformer(func=extract_numeric_password_features, validate=False)), # Стандартизация числовых признаков
    ('classifier', CatBoostClassifier(loss_function='MultiClass',verbose=False))
])

In [ ]:
# Разделение на тренировочную и тестовую выборку
X = df.password
y = df.strength

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,)

#Обучение контейнера на тренировочной выборке
pipeline.fit(X_train,y_train)

#Делаем предсказание тестовых данных обученной нами моделью
y_pred = pipeline.predict(X_test)

#Смотрим на показатели метрики и делаем отчет нашей классификации,чтобы смотреть все измерения метрик в одном блоке
precision = precision_score(y_test, y_pred,average='weighted')
recall = recall_score(y_test, y_pred,average='weighted')
f1 = f1_score(y_test, y_pred,average='weighted')
report = classification_report(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print('')
print(report)

---
**Рассмотрим блок для проверки новых паролей**

---

In [ ]:
#Загрузка датасета в формате csv, по аналогии с форматом из задания
df_test = pd.read_csv('')

# На случай, если в выборке находятся пропущенные значения
df_test_clear = df_test[~(df_test.passwords == '')].copy()


# Предсказание класса паролей из новых данных
y_pred_new = pipeline.predict(df_test_clear.passwords)
y_true = df_test_clear.strength # Истинные значения паролей

#Измерение метрик по сделанным предсказаниям
precision = precision_score(y_true, y_pred,average='weighted')
recall = recall_score(y_true, y_pred,average='weighted')
f1 = f1_score(y_true, y_pred,average='weighted')
report = classification_report(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print('')
print(report)